In [3]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

In [8]:
CUSTOMER_ID = "00000000-6360-4bbb-f78c-b5001f9e006c"

db = SQLDatabase.from_uri("sqlite:///syny.db", sample_rows_in_table_info=3)
context = db.get_context()

llm = ChatOpenAI()
chain = create_sql_query_chain(llm, db)

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "Quantas leituras para gas?"})
response

'SELECT COUNT(*) AS gas_readings_count\nFROM historic_data\nWHERE category = "captured" AND kind = "gas";'

In [19]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

DO NOT list table name or any information about the schema.

Only use the tables below\n\n
CREATE TABLE IF NOT EXISTS "historic_data"(
"device_twin_variable_history_id" TEXT, "device_twin_id" TEXT, "customer_id" TEXT, "device_twin_variable_id" TEXT,
 "name" TEXT, "category" TEXT, "type" TEXT, "kind" TEXT,
 "value" TEXT, "unit" TEXT, "meta" TEXT, "created_at" TEXT,
 "alias" TEXT);

 Some tips on how to query the historic_data table
 To measure consumption aggregate by the value column.

 To measure consumption by utility, filter by the type column.

 To query for a specific time window filter by created_at column.

 To query for a specific customer filter by the customer_id column.
 """

system_message = SystemMessage(content=SQL_PREFIX)

In [20]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

prompt_with_context = chain.get_prompts()[0].partial(table_info=context["table_info"])
print(prompt_with_context.pretty_repr()[:2000])

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [16]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Qual o total de consumo de gas?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5TcFXRgvtPyvNsC7nbjnAqqb', 'function': {'arguments': '{"query":"SELECT SUM(consumo_gas) AS total_consumo_gas FROM table"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 569, 'total_tokens': 597}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6b8789f1-329b-466d-984c-69e36f7398a9-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': 'SELECT SUM(consumo_gas) AS total_consumo_gas FROM table'}, 'id': 'call_5TcFXRgvtPyvNsC7nbjnAqqb'}], usage_metadata={'input_tokens': 569, 'output_tokens': 28, 'total_tokens': 597})]}}
----
{'tools': {'messages': [ToolMessage(content='Error: (sqlite3.OperationalError) near "table": syntax error\n[SQL: SELECT SUM(consumo_gas) AS total_consumo_gas FROM table]\n(Background on this error at: https://sqlalche.me/e/20/e3q